In [203]:
import os
import pynq
import numpy as np
# os.environ['EMCONFIG_PATH'] = os.environ['PWD']
ol=pynq.Overlay("krnl_matmulbertl.xclbin")


In [204]:
from pynq import allocate
Nbanks=8
Nmat=3
Tsize=1024
Nvec=14
# input_buffer = allocate(shape=(Nvec,Tsize), dtype=np.int16, target=ol.HBM14)

In [205]:
low = -15 # -2^15
high =  0# 2^15-1
low, high

(-15, 0)

In [206]:
source_v_np = np.random.randint(low, high, dtype=np.int16, size=((Nmat * Tsize), Nvec))
source_v_np.shape

(3072, 14)

In [207]:
source_w_np = np.random.randint(low, high, dtype=np.int16, size=((Nmat * Tsize),Tsize))
source_w_np.shape

(3072, 1024)

In [208]:
source_w_split_np = []

for i in range(Nbanks):
    source_w_split_np.append(np.zeros((Nmat*Tsize//Nbanks,Tsize)))

In [209]:
source_w_split_np[0].shape

(384, 1024)

In [210]:
# save the split matrices of the reference values for the loading
source_w_split_np = np.vsplit(source_w_np, Nbanks)

In [211]:
outbuf_np = np.zeros((Nvec, Tsize), dtype=np.int32)
outbuf_np.shape

(14, 1024)

In [212]:
outbuf_np = np.matmul(source_v_np.T, source_w_np, dtype=np.int32)
outbuf_np, outbuf_np.shape

(array([[193746, 193610, 193568, ..., 195427, 195659, 194796],
        [198316, 197727, 196957, ..., 200305, 199343, 197307],
        [196071, 197137, 195371, ..., 199620, 200144, 198192],
        ...,
        [189922, 189672, 192438, ..., 193936, 191781, 191424],
        [195122, 193627, 192269, ..., 194655, 195292, 194673],
        [196019, 195608, 193371, ..., 198360, 196971, 195304]], dtype=int32),
 (14, 1024))

In [213]:
Nbanks=8
Nmat=3
Tsize=1024
source_w = [
  pynq.allocate(shape=(Nmat*Tsize//Nbanks,Tsize), dtype=np.int16, target=ol.HBM0),
  pynq.allocate((Nmat*Tsize//Nbanks,Tsize), dtype=np.int16, target=ol.HBM4),
  pynq.allocate((Nmat*Tsize//Nbanks,Tsize), dtype=np.int16, target=ol.HBM8),
  pynq.allocate((Nmat*Tsize//Nbanks,Tsize), dtype=np.int16, target=ol.HBM12),
  pynq.allocate((Nmat*Tsize//Nbanks,Tsize), dtype=np.int16, target=ol.HBM16),
  pynq.allocate((Nmat*Tsize//Nbanks,Tsize), dtype=np.int16, target=ol.HBM20),
  pynq.allocate((Nmat*Tsize//Nbanks,Tsize), dtype=np.int16, target=ol.HBM24),
  pynq.allocate((Nmat*Tsize//Nbanks,Tsize), dtype=np.int16, target=ol.HBM26)]
 

In [214]:
source_w[0].shape, source_w_split_np[0].shape

((384, 1024), (384, 1024))

In [215]:
for i in range(len(source_w)):
#     source_w[i][:] = np.random.randint(-2^15, high=2^15-1, dtype=np.int16, size=(Nmat*Tsize//Nbanks,Tsize))
    source_w[i][:] = source_w_split_np[i]

In [216]:
source_w[0].shape


(384, 1024)

In [217]:
for i in range(8):
    source_w[i].sync_to_device()

In [218]:
source_v = pynq.allocate(((Nmat * Tsize), Nvec), dtype=np.int16, target=ol.HBM14)
print("source_v shape", source_v.shape)
# source_v[:] = np.random.randint(-2^15, high=2^15-1, dtype=np.int16, size=(Tsize,Nvec))
source_v[:] = source_v_np
print(type(source_v))
source_v.sync_to_device()
outbuf = pynq.allocate((Nvec,Tsize), dtype=np.int32, target=ol.HBM14)
print("outbuf shape", outbuf.shape)

source_v shape (3072, 14)
<class 'pynq.buffer.PynqBuffer'>
outbuf shape (14, 1024)


In [219]:
ol.feeder_1.call(
	source_v,
	source_w[0],
	source_w[1],
	source_w[2],
	source_w[3],
	source_w[4],
	source_w[5],
	source_w[6],
	source_w[7],
    outbuf, 0)

outbuf.sync_from_device()

In [220]:
# for i in outbuf[3]:
#     if i != 0: print(i)

In [221]:
outbuf.shape

(14, 1024)

In [222]:
outbuf_np.shape

(14, 1024)

In [223]:
source_v

PynqBuffer([[ -5,  -7,  -2, ...,  -4,  -8, -12],
            [-10,  -6,  -7, ..., -14,  -1,  -9],
            [ -1,  -8,  -5, ...,  -5, -13,  -1],
            ...,
            [-10, -14, -11, ...,  -2,  -5, -14],
            [-14, -14,  -4, ...,  -4, -15,  -6],
            [ -8,  -5, -15, ..., -12,  -4, -12]], dtype=int16)

In [224]:
source_w[0]

PynqBuffer([[-10, -11,  -5, ..., -11,  -7, -11],
            [ -3, -15,  -2, ..., -12, -13,  -1],
            [ -6,  -7, -15, ...,  -2,  -5, -11],
            ...,
            [-15,  -6,  -4, ...,  -3,  -9,  -4],
            [ -6,  -7, -11, ..., -11, -15,  -4],
            [ -3,  -6,  -3, ...,  -3, -12,  -6]], dtype=int16)

In [225]:
outbuf

PynqBuffer([[0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            ...,
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [226]:
outbuf_np

array([[193746, 193610, 193568, ..., 195427, 195659, 194796],
       [198316, 197727, 196957, ..., 200305, 199343, 197307],
       [196071, 197137, 195371, ..., 199620, 200144, 198192],
       ...,
       [189922, 189672, 192438, ..., 193936, 191781, 191424],
       [195122, 193627, 192269, ..., 194655, 195292, 194673],
       [196019, 195608, 193371, ..., 198360, 196971, 195304]], dtype=int32)

In [227]:
np.where(outbuf == 0 )

(array([ 0,  0,  0, ..., 13, 13, 13]),
 array([   0,    1,    2, ..., 1021, 1022, 1023]))